## Curso: Proyecto Integrador
### Integrantes:

#### Andrea Carolina Garza Nacif A01280427
#### Jhamyr Arnulfo Alcalde Oballe A01795409
#### Víctor Arista Beltrán A01012223

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import Markdown, display
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [2]:
DATA_PATH = Path("/Users/andreagarzanacif/Desktop/sparkvista_dataset.csv")
df_raw = pd.read_csv(DATA_PATH)
# Aseguramos que sean string
df_raw['Year Founded'] = df_raw['Year Founded'].astype(str)
df_raw['IPO Date'] = df_raw['IPO Date'].astype(str)

# Extraemos el año de 4 dígitos
df_raw['Year Founded'] = df_raw['Year Founded'].str.extract(r'(\d{4})')
df_raw['IPO Date'] = df_raw['IPO Date'].str.extract(r'(\d{4})')

# Convertimos a numérico
df_raw['Year Founded'] = pd.to_numeric(df_raw['Year Founded'], errors='coerce')
df_raw['IPO Date'] = pd.to_numeric(df_raw['IPO Date'], errors='coerce')

# Convertimos a enteros nullables
df_raw['Year Founded'] = df_raw['Year Founded'].astype('Int64')
df_raw['IPO Date'] = df_raw['IPO Date'].astype('Int64')
#convertir a datetime
df_raw['Year Founded'] = pd.to_datetime(df_raw['Year Founded'], format='%Y', errors='coerce')
df_raw['IPO Date'] = pd.to_datetime(df_raw['IPO Date'], format='%Y', errors='coerce')
print(f"Shape original: {df_raw.shape}")
df_raw.head()

Shape original: (651, 16)


,Megatrend,Area,Firm,Short Description,Keywords,Role of Utility,Website,Twitter Handle,Country,Language,Year Founded,Funding Status,Total Funding Amount (National Currency),Currency,Total Funding Amount (USD),IPO Date
0,Other,Energy Storage,Aceleron,"Develops serviceable, upgradeable, and recycla...","Battery, Circular Economy",NaN,aceleronenergy.com,NaN,United Kingdom,English,2016-01-01,Private,£10.6 million,USD,14098000.0,NaT
1,Clean Energy,Renewable Energy,ACME Group,Indian renewable energy company focusing on so...,"Solar, Wind, Green Hydrogen",NaN,acme.in,NaN,India,English,2003-01-01,Private,₹40 billion,USD,472000000.0,NaT
2,Other,Electric Mobility,Adaptive City Mobility,Developed lightweight electric vehicles with s...,"Electric Vehicles, Urban Mobility",NaN,NaN,NaN,Germany,German,2013-01-01,Insolvent,NaN,USD,NaN,NaT
3,Other,Battery Technology,Addionics,Israeli startup redesigning battery architectu...,"Battery, 3D Electrodes",NaN,addionics.com,NaN,Israel,English,2017-01-01,Private,NaN,USD,NaN,NaT
4,Other,Recycling Technology,Agilyx,Develops advanced recycling technologies to co...,"Recycling, Plastics",NaN,agilyx.com,@agilyx,United States of America,English,2014-01-01,Private,$31M,$31M,NaN,NaT


In [3]:
#Establecer el objetivo de las caracteristicas
TARGET_COL = "Total Funding Amount (USD)" 

drop_cols = [col for col in ["Total Funding Amount (USD)"]
             if col in df_raw.columns]
y = df_raw[TARGET_COL].copy()
X = df_raw.drop(columns=drop_cols).copy()

print("X shape:", X.shape, "| y shape:", y.shape)


X shape: (651, 15) | y shape: (651,)


In [7]:
num_cols = X.select_dtypes(include=['number']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
print("Variables numéricas:", num_cols)
print("Variables categóricas:", cat_cols)

Variables numéricas: []
Variables categóricas: ['Megatrend', 'Area', 'Firm', 'Short Description', 'Keywords', 'Role of Utility', 'Website', 'Twitter Handle', 'Country', 'Language', 'Funding Status', 'Total Funding Amount (National Currency)', 'Currency']


In [8]:
#Inicia analisis avanzado

#Subpipelines
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler())
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe",     OneHotEncoder(
        handle_unknown="ignore",
        sparse_output=False
    ))
])

#Preprocesamos la información
preprocessor = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

#Selección de post caracteristicas
postprocessor = Pipeline([
    ("var_thresh", VarianceThreshold(threshold=0.0)),
    ("pca",        PCA(n_components=0.95, random_state=42))
])

#Pipeline Global
full_pipeline = Pipeline([
    ("prep", preprocessor),
    ("fs",   postprocessor)
])

#Ejecucción
X_ready = full_pipeline.fit_transform(X)
print(f"Shape tras preparación: {X_ready.shape}")


Shape tras preparación: (651, 553)


In [9]:
# ---------------- 4 · GUARDAR MATRIZ PROCESADA (opcional) -------------
out_path = DATA_PATH.with_name("X_prepared.npy")
np.save(out_path, X_ready)
print("Matriz X_ready guardada en:/Users/andreagarzanacif/Desktop/",out_path )


Matriz X_ready guardada en:/Users/andreagarzanacif/Desktop/ /Users/andreagarzanacif/Desktop/X_prepared.npy


In [10]:

vt      = full_pipeline.named_steps["fs"].named_steps["var_thresh"]
pca     = full_pipeline.named_steps["fs"].named_steps["pca"]

varianza_total_inicial = vt.variances_.shape[0]           # Número de variables al inicio
n_variables_utilizadas = vt.get_support().sum()           # Variables retenidas tras umbral de varianza
n_variables_eliminadas = varianza_total_inicial - n_variables_utilizadas  # Variables eliminadas por baja varianza
n_componentes_pca      = pca.n_components_                # Componentes retenidas por PCA              

conclusions = f"""
### Conclusiones


1. **Transformaciones aplicadas**  
   * **One-Hot Encoding** convirtió **{len(cat_cols)}** variables categóricas en un
     espacio numérico sin perder información.  

2. **Reducción de dimensionalidad**  
   * **VarianceThreshold** descartó **{n_variables_eliminadas}** rasgos casi constantes,
     reduciendo ruido y costes de cómputo.  
   * **PCA** comprimió la información en **{n_componentes_pca}** componentes principales
     conservando ≥ 95 % de la varianza total; se mitigó colinealidad y se
     acelerará el entrenamiento de modelos posteriores.

3. **Impacto Futuro**  
   * La salida `X_ready` es un **dataset consistente, compacto y
     escalado**, listo para la fase *Modeling*.  
"""

display(Markdown(conclusions))



### Conclusiones


1. **Transformaciones aplicadas**  
   * **One-Hot Encoding** convirtió **13** variables categóricas en un
     espacio numérico sin perder información.  
   * **Estandarización** normalizó **0** variables numéricas; esto
     evita dominancia de escalas en algoritmos sensibles a la magnitud.

2. **Reducción de dimensionalidad**  
   * **VarianceThreshold** descartó **0** rasgos casi constantes,
     reduciendo ruido y costes de cómputo.  
   * **PCA** comprimió la información en **553** componentes principales
     conservando ≥ 95 % de la varianza total; se mitigó colinealidad y se
     acelerará el entrenamiento de modelos posteriores.

3. **Impacto Futuro**  
   * La salida `X_ready` es un **dataset consistente, compacto y
     escalado**, listo para la fase *Modeling*.  
